<a href="https://colab.research.google.com/github/Takkar-915/movie_review/blob/main/movie_review_logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

映画レビューの内容が肯定的か否定的かの2値分類を行うモデルを作成する。
データセットは(https://ai.stanford.edu/~amaas/data/sentiment/)
に公開されているLarge Movie Review Dataset v1.0を用いた。
特徴量ベクトルを抽出する際、tf_idfモデルを使用し、アルゴリズムはロジスティクス回帰を用いた。

In [ ]:
!pip install pyprind
!pip install nltk

映画レビューデータセットを読み込む

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tarfile

#tarファイルに含まれているファイルをすべて取り出す。
with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
  tar.extractall()

reviewとsentimentのみのデータセットに編集する

In [ ]:
import pyprind
import pandas as pd
import os

basepath = 'aclImdb'

labels = {'pos': 1, 'neg' :0}
#進捗状況を確認
pbar = pyprind.ProgBar(50000)
"""
aclImdbのファイル構成としてtrainフォルダ,testフォルダの下に
それぞれpos,negフォルダがある。
以下のようにパスを結合して、レヴュー文と評価のみを取り出したデータセット
を作成する
"""

df = pd.DataFrame()
for i in ('test','train'):
  for j in ('pos', 'neg'):
    path = os.path.join(basepath,i,j)
    for file in sorted(os.listdir(path)):
      with open(os.path.join(path,file),'r',encoding='utf-8') as infile:
        txt = infile.read()
      df =df.append([[txt,labels[j]]],ignore_index=True)
      pbar.update()
df.columns = ['review','sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:20


上で作成したデータフレームをシャッフル

In [ ]:
import numpy as np

np.random.seed(0)
#渡した配列の要素(df.index)をランダムに並び替える
df = df.reindex(np.random.permutation(df.index))

いったんcsvファイルとして保存

In [ ]:
df.to_csv('movie_data.csv', index = False, encoding='utf-8')

cvsファイルの中身を確認

In [ ]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
#df.head(5)
df.shape

(50000, 2)

ここからは、正規表現を用いてHTMLマークアップ、句読点などの不要な情報を削除

In [ ]:
import re

def preprocessor(text):

  #文字列を置換
  text = re.sub('<[^>]*>', '', text)

  emozi = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)

  text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emozi).replace('-', ''))
  return text

df['review'] = df['review'].apply(preprocessor)

正規表現の結果を確認する

In [ ]:
df.head(5)

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0


テキストデータを分析するために、文書をトークン化する

In [ ]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

#トークン化したものから語幹を取り出す
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

自然言語処理のライブラリを用いる

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


以下より、tf_idfを用いて、レヴュー文を数値の特徴量ベクトルとして表現し、ロジスティクス回帰を行う。

まずは、訓練データとテストデータに分割

In [ ]:
x_train = df.loc[:25000,'review'].values
y_train = df.loc[:25000,'sentiment'].values
x_test = df.loc[25000:,'review'].values
y_test = df.loc[25000:,'sentiment'].values

tf_idfモデル適用

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

In [33]:
gs_lr_tfidf.fit(x_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


KeyboardInterrupt: ignored